In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split


In [3]:
!ls


 data_description.txt			    test.csv
'Exercise_ Bar Charts and Heatmaps.ipynb'   test.csv.gz
'Exercise_ Hello, Seaborn.ipynb'	    train.csv
 exercise-missing-values.ipynb		    train.csv.gz
 sample_submission.csv			    Untitled.ipynb
 sample_submission.csv.gz


In [4]:
!ls

'Exercise_ Bar Charts and Heatmaps.ipynb'   homeprice
'Exercise_ Hello, Seaborn.ipynb'	    Untitled.ipynb
 exercise-missing-values.ipynb


In [26]:
train_df=pd.read_csv('homeprice/train.csv');
test_df =pd.read_csv('homeprice/test.csv');



In [27]:
train_df.dropna(axis=0,subset=['SalePrice'],inplace=True)
y=train_df['SalePrice']

#drop SalePrice
train_df.drop('SalePrice',axis=1,inplace=True)
train_df.shape

(1460, 80)

In [28]:
X=train_df.select_dtypes(exclude=['object'])
t_df=test_df.select_dtypes(exclude=['object'])
print(str(X.shape)+' '+str(t_df.shape))

(1460, 37) (1459, 37)


In [31]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [32]:
print(X_train.shape)

(1168, 37)


In [39]:
miss_val=(X_train.isnull().sum())
miss_val=miss_val[miss_val>0]

In [41]:
num_rows=X_train.shape[0]

num_cols_with_missing=miss_val.shape[0]

tot_missing= miss_val.sum()


276

In [42]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [43]:
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
cols_with_missing

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [44]:
reduced_X_train= X_train.drop(cols_with_missing,axis=1)
reduced_X_valid= X_valid.drop(cols_with_missing,axis=1)
print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))


MAE from Approach 1 (Drop columns with missing values):
17952.591404109586


In [45]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17952.591404109586


In [47]:
from sklearn.impute import SimpleImputer

# Fill in the lines below: imputation
myimpute= SimpleImputer()
imputed_X_train = pd.DataFrame(myimpute.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(myimpute.transform(X_valid))

# Fill in the lines below: imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns



In [48]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE (Imputation):
18250.608013698627


In [50]:
# Imputation
final_imputer = SimpleImputer(strategy='median')
final_X_train = pd.DataFrame(final_imputer.fit_transform(X_train))
final_X_valid = pd.DataFrame(final_imputer.transform(X_valid))

# Imputation removed column names; put them back
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns


# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

MAE (Your approach):
18103.602945205483


In [52]:
# Fill in the line below: preprocess test data
final_X_test = pd.DataFrame(final_imputer.transform(t_df))

# Get test predictions
preds_test = model.predict(final_X_test)


In [54]:
# Save test predictions to file
output = pd.DataFrame({'Id': t_df.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)